
# 📊 Séance 4 – Statistiques descriptives & tests

Dans cette séance, nous allons explorer les relations entre certaines variables de notre dataset `df_clean` à l'aide de tests statistiques. Nous allons aussi vérifier les conditions de validité de chaque test.

# **1. Les basiques**

In [ ]:
# 📦 Chargement des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Télécharger le fichier CSV directement depuis GitHub
!wget https://raw.githubusercontent.com/bidoscar/AFRICITIZEN-ACDS-Coding/main/Seance_4_Statistiques_Descriptive_Tests_Hypotheses/Datasets/df_amazon_clean.csv

In [ ]:
## 1. Chargement et aperçu des données

df = pd.read_csv("df_amazon_clean.csv")
df.head()

# **2. Statistiques descriptives**

In [ ]:
# Croiser les variables numériques entre elles
sns.pairplot(df, hue= 'rating_level')

In [ ]:
# Croiser les variables numériques entre elles
sns.pairplot(df, hue= 'category_level_1')

In [ ]:
df.describe()

In [ ]:
df['rating_level'].value_counts()

In [ ]:
df['category_level_1'].value_counts()

# **3: Analyse des relations**

## **3.1 Prix (`discounted_price`) ~ Note (`rating`)**

In [ ]:
# Verifions si la variable est normalement distribuee
sns.histplot(df['discounted_price'], kde=True, color='green')
plt.show()

🧪 **Test de normalité – Shapiro-Wilk**

But : Vérifier si une variable suit une distribution normale (gaussienne).

➤ Pourquoi c’est important ?
Certains tests statistiques (comme le test de Pearson ou l’ANOVA) présument que les données sont normalement distribuées. Si cette hypothèse est violée, il faut alors utiliser des tests non paramétriques (comme Spearman ou Kruskal-Wallis).

➤ Le test :
H₀ (hypothèse nulle) : Les données suivent une distribution normale.

H₁ : Les données ne suivent pas une distribution normale.

Si la p-value < 0.05, on rejette H₀ → les données ne sont pas normales.

In [ ]:
# Verifions la normalité
from scipy.stats import shapiro

In [ ]:
stat, p = shapiro(df['discounted_price'])
print('stat=%.3f, p=%.3f' % (stat, p))

In [ ]:
# Vérifions le relation
from scipy.stats import pearsonr, spearmanr

**1. pearsonr() – Corrélation de Pearson**

But : Mesurer la relation linéaire entre deux variables quantitatives continues.

✅ Conditions d'application :
* Les deux variables doivent suivre une distribution normale (test Shapiro-Wilk recommandé).

* La relation entre les variables doit être linéaire.

⚙️ Comment ça marche ?
Calcule un coefficient r ∈ [-1, 1]

* r ≈ 1 : forte corrélation positive

* r ≈ -1 : forte corrélation négative

* r ≈ 0 : pas de corrélation linéaire

Retourne aussi une p-value pour tester si cette corrélation est significative.

In [ ]:
pearsonr(df['discounted_price'], df['rating'])  # seulement si normalité

**2. spearmanr() – Corrélation de Spearman**

But : Mesurer la relation monotone (pas forcément linéaire) entre deux variables ordinales ou quantitatives.

✅ Quand l’utiliser ?
* Quand les données ne suivent pas une distribution normale.

* Quand la relation semble non linéaire

* Quand les données contiennent des valeurs extrêmes ou aberrantes.

⚙️ Comment ça marche ?
* Retourne aussi une p-value.

In [ ]:
spearmanr(df['discounted_price'], df['rating']) # si non-normalité

**Et la méthode df.corr() ?**

In [ ]:
df[['discounted_price','rating']].corr()

df.corr() utilise par défaut la corrélation de Pearson, sans vérification de normalité, sans p-value, et sans possibilité de traiter les valeurs manquantes avec autant de finesse.

Elle donne un aperçu rapide mais pas suffisant pour une analyse statistique rigoureuse.

## **3.2 Prix ~ Classe de note (`rating_level`)**

L’ANOVA permet de comparer les moyennes de plusieurs groupes pour voir s’il existe au moins une différence significative.

📌 Hypothèses de base
* Les observations dans chaque groupe doivent être indépendantes les unes des autres.
* Les données de chaque groupe doivent suivre une distribution normale.
* Les variances des groupes doivent être égales.

🧪 **Test d’homogénéité des variances – Levene**

But : Vérifier si plusieurs groupes ont des variances similaires.

➤ Pourquoi c’est important ?
L’ANOVA (test de comparaison des moyennes entre plusieurs groupes) suppose que les groupes comparés ont la même variance. Si ce n’est pas le cas, on doit utiliser un test plus robuste comme Kruskal-Wallis.

➤ Le test :
H₀ : Les variances sont égales entre les groupes.

H₁ : Au moins un groupe a une variance différente.

Si p-value < 0.05, on rejette H₀ → variances différentes.

In [ ]:
from scipy.stats import levene
# stat, p = levene(*groups)
# print('stat=%.3f, p=%.3f' % (stat, p))

In [ ]:
# Supposons 2 groupes
g1 = df[df['rating_level'] == 'High']['discounted_price']
g2 = df[df['rating_level'] == 'Medium']['discounted_price']

In [ ]:
# Normalité
print("Normalité g1:", shapiro(g1).pvalue)
print("Normalité g2:", shapiro(g2).pvalue)

In [ ]:
# Homogénéité des variances
stat, p = levene(g1, g2)
print('stat=%.3f, p=%.3f' % (stat, p))


In [ ]:
# On ne peut pas faire ANOVA
from scipy.stats import kruskal

# Hypothèse nulle H₀	"Toutes les moyennes sont égales"
# Hypothèse alternative H₁	"Au moins une moyenne est différente"

stat, p = kruskal(g1, g2)

print('stat=%.3f, p=%.3f' % (stat, p))

In [ ]:
# Mais si on devrait faire ANOVA
from scipy.stats import f_oneway


# Hypothèse nulle H₀	"Toutes les moyennes sont égales"
# Hypothèse alternative H₁	"Au moins une moyenne est différente"

stat, p = f_oneway(g1, g2)
print('stat=%.3f, p=%.3f' % (stat, p))

## **3.3 Classe de note ~ Catégorie**

**Test du Khi² d’indépendance (chi2_contingency)**

📌 Objectif :
* Tester s’il existe une relation entre deux variables qualitatives (catégorielles).

Hypothèses :
* H₀ (nulle)	Les deux variables sont indépendantes
* H₁ (alternative)	Il existe une dépendance entre les deux variables

Conditions d'application
* Les variables doivent être catégorielles (nominales ou ordinales)

* L’échantillon doit être assez grand (chaque cellule attendue ≥ 5 si possible)

* Les observations doivent être indépendantes

In [ ]:
pd.crosstab(df['rating_level'], df['category_level_1'])

Utiliser un test exact : test de Fisher
* Si la table est 2x2, tu peux utiliser

Fusionner les catégories rares
* Si possible, regroupe les petites catégories (par exemple, combiner officeproducts&other avec une autre catégorie ou créer une catégorie "Autres")

In [ ]:
df['category_level_new'] = df['category_level_1'].replace(
    {'officeproducts&other': 'computers&accessories'})

In [ ]:
pd.crosstab(df['rating_level'], df['category_level_new'])

In [ ]:
from scipy.stats import chi2_contingency
stat, p, dof, expected_freq = chi2_contingency(pd.crosstab(df['rating_level'], df['category_level_new']))
print('stat=%.3f, p=%.3f' % (stat, p))
print(expected_freq)